# 1D diffusion equation

### PDE and boundary conditions
The homogeneous diffusion equation reads

 $$\frac{\partial c}{\partial t} = D \frac{\partial^2 c}{\partial x^2},$$

 where $c$ is the independent variable (displacement, concentration, temperature, etc)
 , $D$ is the characteristic Diffusion coefficient ($m^2/s$).
 
 Written by Ali A. Eftekhari
 Last checked: June 2021
 
 Ported to python by Gavin M. Weir June, 2023


In [ ]:
# Enable interactive plotting
%matplotlib notebook   

In [ ]:
import pyfvtool as pyfvm
from pyfvtool import *
import numpy as np
from scipy.integrate import solve_ivp
from scipy.special import erf

# for animation and visualization
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

In [ ]:
# add a utility function to extract 1D profile
# this may later become a method of CellVariable
def get_CellVariable_profile1D(cv):
    x = np.hstack([cv.domain.facecenters.x[0],
                   cv.domain.cellcenters.x,
                   cv.domain.facecenters.x[-1]])
    phi = np.hstack([0.5*(cv.value[0]+cv.value[1]),
                     cv.value[1:-1],
                     0.5*(cv.value[-2]+cv.value[-1])])
    # The size of the ghost cell is always equal to the size of the 
    # first (or last) cell within the domain. The value at the
    # boundary can therefore be obtained by direct averaging with a
    # weight factor of 0.5.
    return (x, phi)

In [ ]:
Lx = float(50)   # length of domain
Nx = int(20)     # number of cells in domain
# dx = Lx/Nx        # step-size in uniform 1d grid
meshstruct = createMesh1D(Nx, Lx) # 1D domain

# Boundary conditions:  all Neumann boundary condition structure
BC = createBC(meshstruct)
#    left boundary 
BC.left.a = float(0.0)
BC.left.b = float(1.0)
BC.left.c = float(1.0)
#    right boundary
BC.right.a = float(0.0)
BC.right.b = float(1.0)
BC.right.c = float(0.0)

# x_face = meshstruct.facecenters.x   # face positions
x = meshstruct.cellcenters.x   # node positions

# ===== Define the transfer coeffs  ===== #
D_val = float(1.0)   # diffusion coefficient
D = createFaceVariable(meshstruct, D_val)          # uniform diffusion coefficient on the grid
alfa = createCellVariable(meshstruct, float(1.0))  # 

# ===== Define the initial values on the grid ===== #
# u0 = np.abs(np.sin(x_cell/Lx*10*np.pi))

# Concentration on grid nodes (cell centers)
c_old = createCellVariable(meshstruct, float(0.0))   # initial values
c_val = c_old

# Loop parameters
dt = 0.1   # time-step for calculations
final_t = 100.0

# Diffusion coefficient matrix
Mdiff = diffusionTerm(D)

# Right-hand side of the problem
M, RHS = pyfvm.boundary.combineBC(BC, Mdiff, np.zeros((Nx+2,)))

# ========= #
# define the ODE function to solve

def dcdt(t, c):
    # eq: dcdt = D d2c/dx2
    # dcdt = @(t,c)(M*c-RHS);
    global M
    global RHS
    return M*c-RHS

# [t_temp, c_temp] = ode45(dcdt, [0 final_t], internalCells(c_old));

sol = solve_ivp(fun=dcdt, t_span=[0.0, final_t], y0=internalCells(c_old), method='RK45',
                          t_eval = final_t, dense_output=False, vectorized=True)
t_temp, c_temp = sol.t, sol.y

# The analytic solution
c_analytical = 1.0 - erf( x /( 2*np.sqrt(D_val*final_t) ) )

# plot both solutions
plot(x, c_temp[-1,:], '-', x, c_analytical, 'o')


# ========= #
# ========= #

In [ ]:
celleval

# Plotting and visualization of the 1D wave equation solution  

hfig1, ax1 = plt.subplots()

xx, uu = get_CellVariable_profile1D(ui[0])
ax1.plot(xx, uu, 'b-', label='initial value')

xx, uu = get_CellVariable_profile1D(ui[-1])
ax1.plot(xx, uu, 'r-', label='final value')

ax1.set_xlim((0, Lx))
ax1.set_ylim((0.0, 1.0))

ax1.set_xlabel('x')
ax1.set_ylabel('c')
ax1.set_title('1D wave equation solution')
ax1.legend(fontsize=8)

# ========= #

# create a figure to handle the solution animation
#   pre-generate the handles to the axes and the line
hfig2 = plt.figure()
ax2 = plt.axes(xlim=(0, Lx), ylim=(0.0, 1.0))
line, = ax2.plot([], [], 'k-', lw=3, label='Numerical solution')

ax2.set_xlabel('x')
ax2.set_ylabel('c')
ax2.set_title('1D wave equation solution')
ax2.legend(fontsize=8)


def init():
    line.set_data([], [])
    return line, 

def animate(ii):
    # x, y = get_CellVariable_profile1D(ui[ii])

    global ui
    line.set_data(*get_CellVariable_profile1D(ui[ii]))
    return line, 

anim = FuncAnimation(hfig2, animate, init_func=init,
            frames=200, interval=10, blit=True)